In [ ]:
import tkinter as tk
from tkinter import messagebox
import re

class ThaiNumberConverter:
    def __init__(self, root):
        self.root = root
        self.root.title("แปลงตัวเลขเป็นคำภาษาไทย")
        self.units = ["", "หนึ่ง", "สอง", "สาม", "สี่", "ห้า", "หก", "เจ็ด", "แปด", "เก้า"]
        self.tens = ["", "สิบ", "ยี่สิบ", "สามสิบ", "สี่สิบ", "ห้าสิบ", "หกสิบ", "เจ็ดสิบ", "แปดสิบ", "เก้าสิบ"]

        self.create_widgets()

    def create_widgets(self):
        # ส่วนรับข้อมูล
        tk.Label(self.root, text="กรอกจำนวนเงิน:").grid(row=0, column=0, padx=10, pady=10)
        self.entry = tk.Entry(self.root, width=20)
        self.entry.grid(row=0, column=1, padx=10, pady=10)
        self.entry.bind("<KeyRelease>", self.format_number)  # จับเหตุการณ์เมื่อผู้ใช้พิมพ์

        # ปุ่มแปลง
        convert_button = tk.Button(self.root, text="แปลง", command=self.convert)
        convert_button.grid(row=0, column=2, padx=10, pady=10)

        # ส่วนแสดงผล
        self.result_label = tk.Label(self.root, text="ผลลัพธ์: ", font=("Arial", 12))
        self.result_label.grid(row=1, column=0, columnspan=3, padx=10, pady=10)

    def format_number(self, event):
        user_input = self.entry.get().replace(",", "").strip()
        if user_input.isdigit() or (user_input.replace(".", "").isdigit() and user_input.count(".") == 1):
            integer_part, *decimal_part = user_input.split(".")
            formatted_number = "{:,}".format(int(integer_part))
            if decimal_part:
                formatted_number += "." + decimal_part[0]
            self.entry.delete(0, tk.END)
            self.entry.insert(0, formatted_number)

    def number_to_thai(self, num):
        def convert_tens(n):
            ten_part = (n % 100) // 10
            parts = []
            if n >= 100:
                parts.append(self.units[n // 100] + "ร้อย")
                n %= 100
            if n >= 10:
                parts.append(self.tens[n // 10])
                n %= 10
            if n > 0:
                if n == 1 and ten_part > 0:
                    parts.append("เอ็ด")
                elif n == 2 and ten_part > 0:
                    parts.append("ยี่")
                else:
                    parts.append(self.units[n])
            return "".join(parts)

        def convert_integer_part(n):
            if n == 0:
                return "ศูนย์"

            parts = []

            if n >= 1000000:
                parts.append(convert_tens(n // 1000000) + "ล้าน")
                n %= 1000000
            if n >= 1000:
                parts.append(convert_tens(n // 1000) + "พัน")
                n %= 1000
            if n >= 100:
                parts.append(convert_tens(n // 100) + "ร้อย")
                n %= 100
            if n > 0:
                parts.append(convert_tens(n))

            return "".join(parts)

        def convert_decimal_part(n):
            if n > 0 and n <= 99:
                ten_part = n // 10
                unit_part = n % 10
                if unit_part == 0:
                    return self.tens[ten_part] + "สตางค์"
                elif unit_part == 1 and ten_part > 0:
                    return self.tens[ten_part] + "เอ็ด" + "สตางค์"
                else:
                    return self.tens[ten_part] + self.units[unit_part] + "สตางค์"
            return ""

        integer_part = int(num)
        decimal_part = round((num - integer_part) * 100)

        result = convert_integer_part(integer_part) + "บาท"
        if decimal_part > 0:
            result += convert_decimal_part(decimal_part)
        else:
            result += "ถ้วน"
        return result

    def convert(self):
        try:
            user_input = self.entry.get().replace(",", "").strip()
            if not user_input:
                raise ValueError("กรุณากรอกจำนวนเงิน")

            if "." in user_input:
                integer_part, decimal_part = user_input.split(".")
                if len(decimal_part) > 2:
                    raise ValueError("โปรแกรมรับค่าทศนิยมได้สูงสุด 2 ตำแหน่งเท่านั้น (0-99 สตางค์)")

            if not re.match(r"^\d+(\.\d{1,2})?$", user_input):
                raise ValueError("กรุณาป้อนค่าเป็นตัวเลขหรือจำนวนเงินเท่านั้น")

            number = float(user_input)
            if number > 9999999999999.99:
                raise ValueError("โปรแกรมรองรับการแปลงจำนวนเงินสูงสุด 9,999,999,999,999.99 บาท")

            result = self.number_to_thai(number)
            self.result_label.config(text=f"ผลลัพธ์: {result}")
        except ValueError as e:
            messagebox.showerror("ข้อผิดพลาด", str(e))

# เริ่มต้นโปรแกรม
if __name__ == "__main__":
    root = tk.Tk()
    app = ThaiNumberConverter(root)
    root.mainloop()


: 